In [1]:
import requests

import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import missingno as msno
from bs4 import BeautifulSoup as bs

## LA
https://lacdph.shinyapps.io/covid19_surveillance_dashboard/_w_4b1f0220/#shiny-tab-city_case_table
http://publichealth.lacounty.gov/media/coronavirus/locations.htm#vaccinated


## Pasadena
https://www.cityofpasadena.net/public-health/covid-19-dashboard/#vaccinations
2019 pop 141258
10772 cases, 1135 < 18, 4283 18-40, 3447 41-64, 1907 65+
305 deaths, 0 < 18, 4 18-40, 41 41-64, 260 65+

## Long Beach
https://app.powerbigov.us/view?r=eyJrIjoiMDNmNWM3ZjgtNjA2OS00YTEyLThhMjUtNTQyMTU1ZWM3Yjk5IiwidCI6IjMxM2YxMWMzLTQyNjgtNGY2YS04ZDNiLWM3ZTY1MDE4M2U3OCJ9
2019 pop 466776
50855 cases, 2631+5189 < 18, 11522+9781 18-40, 7505+6801+2103 41-64, 2103+1987+991+311 65+
808 deaths, 0 < 18, 58 18-40, 154 41-64, 287+309 65+

In [2]:
url = 'http://publichealth.lacounty.gov/media/coronavirus/locations.htm'
response = requests.get(url)

In [3]:
soup = bs(response.content, 'html.parser')
vax_table = soup.find('div', id='vaccinated').nextSibling.nextSibling

In [4]:
vax_rows = vax_table.find_all('tr')[1:]

vax_neighborhoods = []
vax_counts = []

for row in vax_rows:
    n, c, p = row.find_all('td')

    vax_neighborhoods.append(n.text)
    vax_counts.append(c.text)
vax_df = pd.DataFrame({'neighborhood': vax_neighborhoods, 'vax_count': vax_counts})
vax_df = vax_df.set_index('neighborhood')


vax_df

,vax_count
neighborhood,
City of Agoura Hills,3406
City of Alhambra,11914
City of Arcadia,9674
City of Artesia,2521
City of Avalon,250
...,...
Unincorporated - White Fence Farms,343
Unincorporated - Whittier,677
Unincorporated - Whittier Narrows,<5


In [5]:
df = pd.read_csv('/Users/kevinmarlis/Downloads/LA_County_Covid19_CSA_case_death_table.csv')
df.loc[297,['geo_merge']] = 'Unincorporated - San Francisquito Canyon/Bouquet Canyon'
df = df.rename({'geo_merge':'neighborhood'},axis=1)
df = df.set_index('neighborhood')
df = df.drop(['Unnamed: 0', 'adj_case_rate_final', 'adj_death_rate_final'], axis=1)
df.loc['City of Pasadena'] = [10772, (10772*100000)/141258,305,(305*100000)/141258,141258.0]
df.loc['City of Long Beach'] = [50855, (50855*100000)/466776,808,(808*100000)/466776,466776.0]
df

,cases_final,case_rate_final,deaths_final,death_rate_final,population
neighborhood,,,,,
City of Agoura Hills,956.0,4578.000000,14.0,67.000000,20883.0
City of Alhambra,6568.0,7573.000000,174.0,201.000000,86724.0
City of Arcadia,2555.0,4424.000000,101.0,175.000000,57754.0
City of Artesia,1957.0,11652.000000,54.0,322.000000,16795.0
City of Avalon,14.0,362.000000,0.0,0.000000,3869.0
...,...,...,...,...,...
Unincorporated - Harbor Gateway,0.0,0.000000,0.0,0.000000,1.0
Unincorporated - Placerita Canyon,0.0,0.000000,0.0,0.000000,460.0
Unincorporated - West Chatsworth,0.0,0.000000,0.0,0.000000,12.0


In [6]:
merged_df = pd.concat([df, vax_df], axis=1)
merged_df = merged_df.drop(merged_df[merged_df['cases_final'].isnull()].index)
merged_df = merged_df.drop(merged_df[merged_df['vax_count'] == '<5'].index)
merged_df

,cases_final,case_rate_final,deaths_final,death_rate_final,population,vax_count
City of Agoura Hills,956.0,4578.000000,14.0,67.000000,20883.0,3406
City of Alhambra,6568.0,7573.000000,174.0,201.000000,86724.0,11914
City of Arcadia,2555.0,4424.000000,101.0,175.000000,57754.0,9674
City of Artesia,1957.0,11652.000000,54.0,322.000000,16795.0,2521
City of Avalon,14.0,362.000000,0.0,0.000000,3869.0,250
...,...,...,...,...,...,...
Unincorporated - Willowbrook,5777.0,16547.000000,80.0,229.000000,34913.0,1700
Unincorporated - Wiseburn,485.0,8047.000000,8.0,133.000000,6027.0,609
Unincorporated - Placerita Canyon,0.0,0.000000,0.0,0.000000,460.0,17
City of Pasadena,10772.0,7625.762789,305.0,215.916975,141258.0,23956


In [7]:
merged_df['vax_count'] = merged_df['vax_count'].astype('float64')
merged_df['vax_rate'] = (merged_df['vax_count']*100000)/merged_df['population']

In [8]:
cols = {col:col.replace("_final", "") for col in merged_df.columns.tolist()}
merged_df = merged_df.rename(columns=cols)
cols = ['population', 'cases', 'case_rate', 'deaths', 'death_rate', 'vax_count', 'vax_rate']
merged_df = merged_df[cols]
merged_df

,population,cases,case_rate,deaths,death_rate,vax_count,vax_rate
City of Agoura Hills,20883.0,956.0,4578.000000,14.0,67.000000,3406.0,16309.917157
City of Alhambra,86724.0,6568.0,7573.000000,174.0,201.000000,11914.0,13737.834971
City of Arcadia,57754.0,2555.0,4424.000000,101.0,175.000000,9674.0,16750.354954
City of Artesia,16795.0,1957.0,11652.000000,54.0,322.000000,2521.0,15010.419768
City of Avalon,3869.0,14.0,362.000000,0.0,0.000000,250.0,6461.617989
...,...,...,...,...,...,...,...
Unincorporated - Willowbrook,34913.0,5777.0,16547.000000,80.0,229.000000,1700.0,4869.246413
Unincorporated - Wiseburn,6027.0,485.0,8047.000000,8.0,133.000000,609.0,10104.529617
Unincorporated - Placerita Canyon,460.0,0.0,0.000000,0.0,0.000000,17.0,3695.652174
City of Pasadena,141258.0,10772.0,7625.762789,305.0,215.916975,23956.0,16959.039488


In [10]:
merged_df.to_pickle('neighborhood_covid_df')

# Load in CES data

In [43]:
import pickle5 as pickle
with open('CES_neighborhood_df.pkl', "rb") as fh:
    ces_df = pickle.load(fh)
# tracts_df.drop([t for t in tracts_df.index.to_list() if t not in merged_df.index.to_list()])
ces_df

,ozone,pm,diesel,traffic_density,pollution_score,asthma_rate,population_char_score,children_percent,elderly_percent,tract_lists
neighborhood,,,,,,,,,,
,0.044214,11.525000,33.667970,759.405625,5.535631,36.124375,2.958006,6.750000,13.356250,"[6037297601, 6037577504, 6037621326, 603762132..."
City of Agoura Hills,0.051865,9.536303,7.881444,1200.050000,4.629887,19.893333,2.959783,10.833333,11.266667,"[6037800332, 6037800327, 6037800324]"
City of Alhambra,0.049204,11.525000,22.743278,1382.278500,6.823814,31.803500,4.868749,9.980000,14.590000,"[6037481606, 6037481605, 6037480304, 603748080..."
City of Arcadia,0.051412,10.485542,12.843261,1135.541818,5.588927,16.549091,2.984220,9.972727,16.245455,"[6037430801, 6037430724, 6037430701, 603743170..."
City of Artesia,0.044312,11.210000,23.684496,1311.766667,5.672172,34.370000,5.959942,11.666667,12.766667,"[6037554802, 6037554900, 6037554801]"
...,...,...,...,...,...,...,...,...,...,...
Unincorporated - White Fence Farms,0.062163,6.182866,6.014267,737.770000,4.550852,38.490000,4.557277,10.600000,12.800000,[6037910202]
Unincorporated - Whittier,0.046178,12.050000,21.301501,1307.120000,6.867540,63.520000,4.339597,10.000000,13.950000,"[6037501501, 6037500300]"
Unincorporated - Whittier Narrows,0.046178,12.050000,22.470000,1920.210000,8.811509,54.750000,5.523925,13.000000,13.600000,[6037433802]


In [49]:
combined_df = pd.concat([merged_df, ces_df], axis=1)
combined_df

,population,cases,case_rate,deaths,death_rate,vax_count,vax_rate,ozone,pm,diesel,traffic_density,pollution_score,asthma_rate,population_char_score,children_percent,elderly_percent,tract_lists
City of Agoura Hills,20883.0,956.0,4578.0,14.0,67.0,3406.0,16309.917157,0.051865,9.536303,7.881444,1200.050000,4.629887,19.893333,2.959783,10.833333,11.266667,"[6037800332, 6037800327, 6037800324]"
City of Alhambra,86724.0,6568.0,7573.0,174.0,201.0,11914.0,13737.834971,0.049204,11.525000,22.743278,1382.278500,6.823814,31.803500,4.868749,9.980000,14.590000,"[6037481606, 6037481605, 6037480304, 603748080..."
City of Arcadia,57754.0,2555.0,4424.0,101.0,175.0,9674.0,16750.354954,0.051412,10.485542,12.843261,1135.541818,5.588927,16.549091,2.984220,9.972727,16.245455,"[6037430801, 6037430724, 6037430701, 603743170..."
City of Artesia,16795.0,1957.0,11652.0,54.0,322.0,2521.0,15010.419768,0.044312,11.210000,23.684496,1311.766667,5.672172,34.370000,5.959942,11.666667,12.766667,"[6037554802, 6037554900, 6037554801]"
City of Avalon,3869.0,14.0,362.0,0.0,0.0,250.0,6461.617989,0.040127,NaN,14.876867,NaN,3.978593,49.390000,4.783565,12.600000,11.300000,[6037599000]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Los Angeles - Angeles National Forest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.055122,10.790000,4.086115,76.560000,3.378989,30.960000,NaN,0.000000,10.000000,[6037980026]
Unincorporated - South Edwards,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.062163,5.763686,0.267263,106.110000,2.592780,44.270000,NaN,0.000000,100.000000,[6037980003]
Unincorporated - Universal City,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.053109,12.050000,20.950000,3359.090000,8.168672,0.000000,NaN,0.000000,0.000000,[6037320000]
Unincorporated - West Chatsworth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.055122,9.955483,2.734419,282.620000,3.021543,52.490000,4.308070,8.900000,11.700000,[6037134424]


In [78]:
(combined_df.elderly_percent - (combined_df.vax_rate/1000)).idxmin()

'City of Industry'

In [79]:
combined_df.to_csv('Neighborhood_CES_Covid.csv')